# Building Your Predictor

Forecasting is used in a variety of applications and business use cases: For example, retailers need to forecast the sales of their products to decide how much stock they need by location, Manufacturers need to estimate the number of parts required at their factories to optimize their supply chain, Businesses need to estimate their flexible workforce needs, Utilities need to forecast electricity consumption needs in order to attain an efficient energy network, and enterprises need to estimate their cloud infrastructure needs.

<img src="https://amazon-forecast-samples.s3-us-west-2.amazonaws.com/common/images/forecast_overview_steps.png" width="98%">

In this notebook we will be walking through the steps outlined in 2nd-through-4th boxes above to build and query your first forecast.


## Table Of Contents
* Step 1: [Setup Amazon Forecast](#setup)
* Step 2: [Create a Predictor](#createPredictor)
* Step 3: [Get Predictor Error Metrics from Backtesting](#predictorErrors)
* Step 4: [Create a Forecast](#createForecast)
* Step 5: [Query the Forecast](#queryForecast)
* [Next Steps](#nextSteps)

For more informations about APIs, please check the [documentation](https://docs.aws.amazon.com/forecast/latest/dg/what-is-forecast.html)

## Step 1: Setup Amazon Forecast<a class="anchor" id="setup"></a>


This section sets up the permissions and relevant endpoints.

In [1]:
import sys
import os
import time
import pandas as pd

# importing forecast notebook utility from notebooks/common directory
sys.path.insert( 0, os.path.abspath("./common") )
import util

%reload_ext autoreload
import boto3
import s3fs

The line below will retrieve your stored variables from the first notebook.

In [2]:
%store -r

# Print your choices from first notebook
# print(f"item_id = {item_id}")
print(f"project = {PROJECT}")
print(f"data_version = {DATA_VERSION}")
print(f"Forecast length = {FORECAST_LENGTH}")
print(f"Dataset frequency = {DATASET_FREQUENCY}")
print(f"Timestamp format = {TIMESTAMP_FORMAT}")
print(f"dataset_group_arn = {dataset_group_arn}")
print(f"role_arn = {role_arn}")
%store -r bucket_name
print(f"bucket_name = {bucket_name}")
%store -r region
print(f"region = {region}")

project = m5_sku_prediction_2m
data_version = 4
Forecast length = 2
Dataset frequency = M
Timestamp format = yyyy-MM-dd
dataset_group_arn = arn:aws:forecast:us-west-2:230755935769:dataset-group/m5_sku_prediction_2m_4
role_arn = arn:aws:iam::230755935769:role/ForecastNotebookRole
bucket_name = sagemaker-us-west-2-230755935769
region = us-west-2


The last part of the setup process is to validate that your account can communicate with Amazon Forecast, the cell below does just that.

In [3]:
# Connect API session
session = boto3.Session(region_name=region) 
forecast = session.client(service_name='forecast') 
forecastquery = session.client(service_name='forecastquery')

## Step 2: Create a Predictor <a class="anchor" id="createPredictor"></a>

Once the datasets are specified with the corresponding schema, Amazon Forecast will automatically aggregate, at the specified time granularity, all the relevant pieces of information for each item, such as sales, price, promotions, as well as categorical attributes, and generate the desired dataset. Next, one can choose an algorithm (forecasting model) and evaluate how well this particular algorithm works on this dataset. The following graph gives a high-level overview of the forecasting models.
<img src="https://amazon-forecast-samples.s3-us-west-2.amazonaws.com/common/images/recipes.png" width="98%">
<img src="https://amazon-forecast-samples.s3-us-west-2.amazonaws.com/common/images/mqcnn.png" width="70%">
<img src="https://amazon-forecast-samples.s3-us-west-2.amazonaws.com/common/images/pred_details.png">


Amazon Forecast provides several state-of-the-art forecasting algorithms including classic forecasting methods such as ETS, ARIMA, Prophet and deep learning approaches such as DeepAR+. Classical forecasting methods, such as Autoregressive Integrated Moving Average (ARIMA) or Exponential Smoothing (ETS), fit a single model to each individual time series, and then use that model to extrapolate the time series into the future. Amazon's Non-Parametric Time Series (NPTS) forecaster also fits a single model to each individual time series.  Unlike the naive or seasonal naive forecasters that use a fixed time index (the previous index $T-1$ or the past season $T - \tau$) as the prediction for time step $T$, NPTS randomly samples a time index $t \in \{0, \dots T-1\}$ in the past to generate a sample for the current time step $T$.

In many applications, you may encounter many similar time series across a set of cross-sectional units. Examples of such time series groupings are demand for different products, server loads, and requests for web pages. In this case, it can be beneficial to train a single model jointly over all of these time series. CNN-QR and DeepAR+ take this approach, outperforming the standard ARIMA and ETS methods when your dataset contains hundreds of related time series. The trained model can also be used for generating forecasts for new time series that are similar to the ones it has been trained on. 

While deep learning approaches can outperform standard methods, this is only possible when there is sufficient data available for training. It is not true for example when one trains a neural network with a time-series containing only a few dozen observations. Amazon Forecast provides the best of two worlds allowing users to either choose a specific algorithm or let Amazon Forecast automatically perform model selection. 


## How to evaluate a forecasting model?

Before moving forward, let's first introduce the notion of *backtest* when evaluating forecasting models. The key difference between evaluating forecasting algorithms and standard ML applications is that we need to make sure there is no future information gets used in the past. In other words, the procedure needs to be causal. 

<img src="https://amazon-forecast-samples.s3-us-west-2.amazonaws.com/common/images/backtest.png" width=70%>



In [4]:
# Which algorithm do you want to use?  Choices are:
# 1. Choose PerformAutoML=True if you want to let Amazon Forecast choose a recipe automatically.  
# 2. If you know which recipe you want, the next level of automation is PerformHPO=True.
# 3. Finally, you can specify exactly which recipe and enter your own hyperparameter values
# https://docs.aws.amazon.com/forecast/latest/dg/aws-forecast-choosing-recipes.html

algorithm_arn_prefix = 'arn:aws:forecast:::algorithm/'
algorithm = 'CNN-QR'
algorithm_arn = algorithm_arn_prefix + algorithm
predictor = f"{PROJECT}_{DATA_VERSION}"
print(f"Predictor Name = {predictor}")

Predictor Name = m5_sku_prediction_2m_4


In [5]:
create_predictor_response = \
    forecast.create_auto_predictor(PredictorName=predictor, 
                                 ForecastHorizon=FORECAST_LENGTH,
                                 ForecastFrequency=DATASET_FREQUENCY, 
                                 OptimizationMetric='RMSE',
                                 ExplainPredictor=True,  
                                 ForecastDimensions=['store_id', 'state_id'],
                                 DataConfig={
                                        'DatasetGroupArn': dataset_group_arn,
                                        'AttributeConfigs': [
                                            {
                                                'AttributeName': "demand",
                                                'Transformations': {
                                                    "aggregation": "sum",
                                                    "backfill": "zero",
                                                    "frontfill": "none",
                                                    "middlefill": "zero"
                                                }
                                            },
                                    
#                                             {
#                                                 'AttributeName': "rolling_mean_t4",
#                                                 'Transformations': {
#                                                     "backfill": "mean",
#                                                     "futurefill": "mean",
#                                                     "middlefill": "mean"
#                                                 }
#                                             },
#                                             {
#                                                 'AttributeName': "rolling_std_t4",
#                                                 'Transformations': {
#                                                     "backfill": "mean",
#                                                     "futurefill": "mean",
#                                                     "middlefill": "mean"
#                                                 }
#                                             },
#                                             {
#                                                 'AttributeName': "rolling_mean_t12",
#                                                 'Transformations': {
#                                                     "backfill": "mean",
#                                                     "futurefill": "mean",
#                                                     "middlefill": "mean"
#                                                 }
#                                             },
#                                             {
#                                                 'AttributeName': "rolling_mean_t24",
#                                                 'Transformations': {
#                                                     "backfill": "mean",
#                                                     "futurefill": "mean",
#                                                     "middlefill": "mean"
#                                                 }
#                                             },
                                        ]
                                    
                                    }
                                )  

                                


In [6]:
predictor_arn = create_predictor_response['PredictorArn']

Check the status of the predictor. When the status change from **CREATE_IN_PROGRESS** to **ACTIVE**, we can continue to next steps. Depending on data size, model selection and choice of hyper parameters tuning，it can take several hours to be **ACTIVE**.

In [ ]:
# status = util.wait(lambda: forecast.describe_predictor(PredictorArn=predictor_arn))
status = util.wait(lambda: forecast.describe_auto_predictor(PredictorArn=predictor_arn))
assert status

CREATE_PENDING ..
CREATE_IN_PROGRESS ....................................................................................................................................................................................................................................................................................................................................

In [ ]:
# forecast.describe_predictor(PredictorArn=predictor_arn)
forecast.describe_auto_predictor(PredictorArn=predictor_arn)

In [ ]:
%store predictor_arn
%store bucket_name
%store region 

## Step 3. Get Predictor Error Metrics from Backtesting <a class="anchor" id="predictorErrors"></a>

After creating the predictors, we can query the forecast accuracy given by the backtest scenario and have a quantitative understanding of the performance of the algorithm. Such a process is iterative in nature during model development. When an algorithm with satisfying performance is found, the customer can deploy the predictor into a production environment, and query the forecasts for a particular item to make business decisions. The figure below shows a sample plot of different quantile forecasts of a predictor.

In [ ]:
error_metrics = forecast.get_accuracy_metrics(PredictorArn=predictor_arn)
error_metrics

## Step 4. Create a Forecast <a class="anchor" id="createForecast"></a>

In [ ]:

forecast_name = predictor_arn.split('/')[-1]+"_forecast"

In [ ]:
forecast_name

In [ ]:


create_forecast_response = \
    forecast.create_forecast(ForecastName=forecast_name[0:63],
                             PredictorArn=predictor_arn)

forecast_arn = create_forecast_response['ForecastArn']

Check the status of the forecast process, when the status change from **CREATE_IN_PROGRESS** to **ACTIVE**, we can continue to next steps. Depending on data size, model selection and choice of hyper parameters tuning，it can take several hours to be **ACTIVE**.

In [ ]:
status = util.wait(lambda: forecast.describe_forecast(ForecastArn=forecast_arn))
assert status

In [ ]:
forecast.describe_forecast(ForecastArn=forecast_arn)

In [ ]:
forecast_arn

## Step 5:  Query the Forecast <a class="anchor" id="queryForecast"></a>

Once created, the forecast results are ready and you view them. 

In [ ]:
item_id = 'TWS0926300'

In [ ]:
forecast_response_deep = forecastquery.query_forecast(
    ForecastArn=forecast_arn,
    Filters={"item_id": item_id})

forecast_response_deep

In [ ]:
%store forecast_arn
%store predictor_arn

## Next Steps<a class="anchor" id="nextSteps"></a>

Congratulations!! You've trained your first Amazon Forecast model and generated your first forecast!!

To dive deeper, here are a couple options for further evaluation:
<ul>
    <li><b>To see an example of single item evaluation in a notebook</b>, see  `3.Evaluating_Your_Predictor.ipynb`.</li>
    <li><b>For an example how to use a notebook and Predictor Backtest Forecasts to evaluate all items at once using custom metrics</b>, see `../advanced/Item_Level_Accuracy/Item_Level_Accuracy_Using_Bike_Example.ipynb`. </li>
    <li><b>Finally, for a production-level example, how to use Amazon QuickSight to visualize either Predictor Backtest Forecasts and/or Forecasts so you can share and socialize the results with others</b>, <a href="https://aws.amazon.com/solutions/implementations/improving-forecast-accuracy-with-machine-learning/?did=sl_card&trk=sl_card" target="_blank">see our automation solution Improving Forecast Accuracy</a></li>
        <li><a href="https://console.aws.amazon.com/cloudformation/home?region=us-east-1#/stacks/new?stackName=forecast-stack&t[…]acy-with-machine-learning-demo.template" target="_blank">Quick launch link for above automation</a></li>
    </ul>
    
For other advanced topics, see the `advanced` section of our notebooks.  Several you may want to check out next:
<ul>
    <li>Example how to use Related Data: <a href="https://github.com/aws-samples/amazon-forecast-samples/blob/master/notebooks/advanced/Incorporating_Related_Time_Series_dataset_to_your_Predictor/Incorporating_Related_Time_Series_dataset_to_your_Predictor.ipynb" target="_blank">Incorporating Related Time Series</a></li>
    <li>Example how to use our built-in, hosted-by-AWS weather data: <a href="https://github.com/aws-samples/amazon-forecast-samples/blob/master/notebooks/advanced/Weather_index/1.%20Training%20your%20model%20with%20Weather%20Index.ipynb" target="_blank">Training your model with Weather Index </a></li>
    </ul>